In [19]:
import openpyxl
import numpy as np
from openpyxl import load_workbook
import pandas as pd

In [9]:
def show_info(x,show_values = False):
    length = 'no length'
    keys = 'not a dict'
    values = 'not a dict'
    shape = 'not a ndarray'
    size = 'not a ndarray'
    if hasattr(x,'__len__'):
        length = len(x)
    if isinstance(x,dict):
        keys = x.keys()
        values = x.values()
    if type(x) is np.ndarray:
        shape = x.shape
        size = x.size
    print(f'type:{type(x)} \nlen:{length}\nshape:{shape}\nsize:{size}\nkeys:{keys}\noriginal info:{x}')
    if show_values:
        print(f'\nvalues:{values}')

## 提取下一步需要的uuid

In [37]:
from dis import show_code


wb = load_workbook('../config/blca.xlsx')
ws = wb['Sheet1']
dir_uuid = ws['A']
included_svs_uuid = []  #最终用于切片的uuid列表
index = []
included_file_svs = [] #与uuid对应的file_svs列表
for i,cell in enumerate(dir_uuid[1:]):
    if cell.fill.fgColor.rgb == 'FFFFFF00':  #这个函数只能读取标准色
        included_svs_uuid.append(cell.value)
        index.append(i)
for i in index:
    included_file_svs.append(ws['B'][1:][i].value)

f:\miniconda\envs\data_analysis\lib\site-packages\openpyxl\reader\excel.py:228: UserWarning: Data Validation extension is not supported and will be removed
  ws_parser.bind_all()


## 将TMB与uuid对应起来

In [48]:
pan_TMB = pd.read_csv('../config/TCGA TMB.csv')
# BLCA_TMB = pan_TMB[pan_TMB['']]
BLCA_TMB = pan_TMB[pan_TMB['Cohort'] == 'BLCA']
BLCA_TMB['TMB_H/L'] = 'L'
for i,tmb in BLCA_TMB['TMB'].items():
    if tmb >= 10:
        BLCA_TMB['TMB_H/L'][i] = 'H'
BLCA_TMB

C:\Users\Doc Wong\AppData\Local\Temp\ipykernel_9020\366613547.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BLCA_TMB['TMB_H/L'] = 'L'
C:\Users\Doc Wong\AppData\Local\Temp\ipykernel_9020\366613547.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  BLCA_TMB['TMB_H/L'][i] = 'H'


,Cohort,ID,Tumor_Sample_ID,Silent per Mb,TMB,TMB_H/L
91,BLCA,TCGA-FD-A43S,TCGA-FD-A43S-01,0.000000,0.000000,L
92,BLCA,TCGA-XF-AAN8,TCGA-XF-AAN8-01,0.030943,0.000000,L
93,BLCA,TCGA-4Z-AA7N,TCGA-4Z-AA7N-01,0.310769,0.310769,L
94,BLCA,TCGA-DK-A3WY,TCGA-DK-A3WY-01,0.089070,0.326590,L
95,BLCA,TCGA-XF-AAME,TCGA-XF-AAME-01,0.063186,0.410711,L
...,...,...,...,...,...,...
495,BLCA,TCGA-DK-A3WW,TCGA-DK-A3WW-01,10.215587,29.433306,H
496,BLCA,TCGA-DK-A1AC,TCGA-DK-A1AC-01,16.737009,37.477656,H
497,BLCA,TCGA-YC-A89H,TCGA-YC-A89H-01,19.620691,40.105187,H
498,BLCA,TCGA-MV-A51V,TCGA-MV-A51V-01,11.779311,40.586437,H


## 将uuid、file_svs写进csv

In [51]:
svs_with_labels = pd.DataFrame({
    'dir_uuid':included_svs_uuid,
    'file_svs':included_file_svs})

def get_sampleID(string):
    sampleID = string[:15]
    return sampleID

svs_with_labels['file_svs'] = svs_with_labels['file_svs'].apply(get_sampleID)
svs_with_labels


,dir_uuid,file_svs
0,d2e43ec6-5027-4f2c-932b-28a681da7cd9,TCGA-2F-A9KO-01
1,bff29d20-3a8f-4a5d-a2de-0e142390551d,TCGA-2F-A9KP-01
2,edcf4ae6-c985-40ad-aff4-a0ce31b46aeb,TCGA-2F-A9KP-01
3,e20c2176-2b57-4cc9-a68a-eb6933bf60b1,TCGA-2F-A9KQ-01
4,a085fe18-1709-417a-a779-fe69f5964766,TCGA-2F-A9KR-01
...,...,...
381,6d94e519-6dea-42d0-a505-f36ab27f2f3f,TCGA-ZF-AA54-01
382,16c68e40-fc90-495a-b856-fe1f11f82143,TCGA-ZF-AA58-01
383,ff7d6bdf-b564-41e0-b3e5-2a649423796b,TCGA-ZF-AA5H-01
384,f7db97bf-bcb4-4383-afe2-36f549a0bdf0,TCGA-ZF-AA5N-01
